# 実行環境の作成

In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py39_4.12.0-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py39" --user

In [2]:
!python --version

Python 3.9.12


## Google Cloud Storage

下記コードでGCPに接続

In [ ]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [ ]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

2022/09/14 10:45:58.350847 Start gcsfuse/0.41.6 (Go version go1.18.4) for app "" using mount point: /content/statistics-hyogo
2022/09/14 10:45:58.362225 Opening GCS connection...
2022/09/14 10:46:00.498213 Mounting file system "statistics-hyogo"...
2022/09/14 10:46:00.498876 File system has been successfully mounted.


## Google Secret Maneger

ライブラリをインストール

In [ ]:
pip install google-cloud-secret-manager

シークレットを呼び出す関数

In [ ]:
from google.cloud import secretmanager

def access_secret(project_id, secret_name, version='latest'):
    client = secretmanager.SecretManagerServiceClient()
    name = client.secret_version_path(project_id, secret_name, version)
    response = client.access_secret_version(request={"name":name})
    payload = response.payload.data.decode("UTF-8")
    return payload

ESTAT_APPIDを取得する

In [ ]:
PROJECT_ID = 'primal-buttress-342908'
ESTAT_APPID = access_secret(PROJECT_ID,'ESTAT_APPID' )
# print(ESTAT_APPID)

## pythonのライブラリ追加

In [ ]:
import urllib.parse
import urllib.request
import json
import pandas as pd

# estatの統計情報を取得

統計表を指定してメタ情報を取得する

[API仕様](https://www.e-stat.go.jp/api/api-info/e-stat-manual3-0#api_2_2)

'statsDataId'に統計表コードを指定

In [ ]:
statsDataId = '0000010101'

メタ情報を取得する関数を作成

In [ ]:
def get_estat_meta(statsDataId):
  # リクエストパラメータ
  p={}
  p['statsDataId'] = statsDataId
  p['appId'] = ESTAT_APPID

  url = 'http://api.e-stat.go.jp/rest/3.0/app/json/getMetaInfo?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

meta = get_estat_meta(statsDataId)
print(meta)

NameError: ignored

## カテゴリコード一覧

カテゴリコード'cat01'のリストを作成する関数

In [ ]:
def get_estat_catefory_codes(statsDataId):
  meta= get_estat_meta(statsDataId)
 
  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

  # カテゴリ一覧
  categories = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']

  # カテゴリコード一覧
  catagory_codes = [d.get('@code') for d in categories]

  return catagory_codes

category_codes = get_estat_catefory_codes(statsDataId)
print(category_codes)
# print(len(category_codes))

NameError: ignored

## 地域コード一覧

地域コード'area'のリスト一覧を作成する関数

全国'00000'は除外している

In [ ]:
def get_estat_area_codes(statsDataId):
  # estat-APIのメタ情報を取得
  meta= get_estat_meta(statsDataId)

  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

  # 地域一覧
  areas = next((d for d in CLASS_OBJ if d['@id'] == 'area'), None)['CLASS']

  # 地域コード一覧（全国'00000'は削除）
  area_codes = [d.get('@code') for d in areas]
  area_codes.remove('00000')

  return area_codes

area_codes = get_estat_area_codes(statsDataId)
print(area_codes)

# estat-APIのデータを取得

まずは、総人口のデータを取得してみる

cat01に'A1101'を指定
areaは全都道府県を指定

リストを指定する場合はカンマ区切りの文字列に変換する

## estat-APIのデータ取得

In [ ]:
params = {}

params['statsDataId'] = statsDataId
params['cdCat01'] = 'A1231'
params['cdArea']=",".join(area_codes)

# print (params)

estat-APIのデータを取得する関数を作成

In [ ]:
def get_estat_data(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

res = get_estat_data(params)
# print(res)

## pandas DataFrameに変換

estat-APIのレスポンスからVALUEを抜き出し、DataFrameに変換する

In [ ]:
res = get_estat_data(params)

# VALUE
VALUE = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

df_value = pd.json_normalize(VALUE)
print(df_value)

カテゴリ情報は'CLASS_OBJ'に格納されている。

これもDataFrameに変換

In [ ]:
# CLASS_INF
CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

categories = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']

df_categories = pd.json_normalize(categories)
print(df_categories)

統計表情報は'TABLE_INF'に格納されている

In [ ]:
# TABLE_INF
TABLE_INF = res['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']

statDataId = TABLE_INF['@id']
statDataName = TABLE_INF['STAT_NAME']['$']

print(statDataId,statDataName)

estat-APIのレスポンスをひとつのDataFrameにまとめる関数を作成

In [ ]:
def conv_estat_response_to_dataframe(response):
  # CLASS_INF
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

  # TABLE_INF
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']

  # VALUE
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

  # VALUEをDataFrameに変換
  df_res = pd.json_normalize(VALUE)

  # CLASS_OBJのDataFrameを結合
  for d in CLASS_OBJ :
    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    key_name = '@{}'.format(d['@id'])
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df_res = pd.merge(df_res, df_class, on=key_name, how='outer')

  # 統計情報を追加
  df_res['statsDataId'] = TABLE_INF['@id']
  df_res['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df_res

df_org = conv_estat_response_to_dataframe(res)
df_org

## DataFrameの整形

使い勝手が良くなるように、DataFrameを整形していく

必要な列だけ抽出する

In [ ]:
df_1 = df_org[['statsDataId','statsDataName','@cat01','@cat01_name','@time','@time_name','@area','@area_name','$','@unit']]

# print(df_1)

列名を変更する

In [ ]:
columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','@area_name':'areaName','$':'value','@unit':'unit'}
df_1 = df_1.rename(columns=columns)

# print(df_1)

欠損値が含まれる行を削除する

In [ ]:
df_1 = df_1.dropna()

# print(df_1)

categoryNameから不要な情報（categoryCode）を削除する

In [ ]:
df_1['categoryName'] = df_1.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

# print(df_1)

timeCodeを4桁文字列に置換する

In [ ]:
df_1['timeCode'] = df_1.apply(lambda x: x['timeCode'][:4], 1)

# print(df_1)

ここまでを関数にまとめる

In [ ]:
def  format_estat_dataframe(df_arg):
  # 必要な列だけ抽出
  df_res =  df_arg[['statsDataId','statsDataName','@cat01','@cat01_name','@time','@time_name','@area','@area_name','$','@unit']]

  # 列名の変更
  columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','@area_name':'areaName','$':'value','@unit':'unit'}
  df_res = df_res.rename(columns=columns)

  # 欠損データ削除
  df_res = df_res.dropna()

  # categoryNameから不要な情報（categoryCode）を削除
  df_res['categoryName'] = df_res.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  # timeCodeを文字列4桁に置換
  df_res['timeCode'] = df_res.apply(lambda x: x['timeCode'][:4], 1)

  return df_res

df_fmt = format_estat_dataframe(df_org)
df_fmt

## 順位を付与する

年度ごとに順位を付与

In [ ]:
def format_estat_dataframe_withrank(df_arg):

  # 年度リストを作成
  times = df_arg['timeCode'].tolist()
  times = list(set(times))

  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  # 年度毎に順位を付与
  for time in times:

    # 単年度のdataを抽出
    df_time = df_arg.copy()[df_arg['timeCode'] == time]

    # valueを数値に変換して、'rank'を付与
    df_time = df_time.astype({'value': float})
    df_time['rank'] = df_time.rank(ascending=False)['value'].astype(int).astype(str)

    # 結合
    df_res = pd.concat([df_res, df_time])

  return df_res

df_last = format_estat_dataframe_withrank(df_fmt)
df_last

## まとめ

paramsを指定してから、順位を付与したDataFrameを取得するまでを関数にまとめる

In [ ]:
params = {}

params['statsDataId'] = statsDataId
params['cdCat01'] = 'A1101'
params['cdArea']=",".join(area_codes)

In [ ]:
def get_estat_dataframe(params):
  # estat-APIのレスポンス取得
  res = get_estat_data(params)

  # レスポンスをDataFrameに変換
  df = conv_estat_response_to_dataframe(res)

  # DataFrameを整形
  df = format_estat_dataframe(df)

  # 順位を付与
  df = format_estat_dataframe_withrank(df)

  return df

df = get_estat_dataframe(params)
# df

In [ ]:
# プロジェクトの定義
project_id='primal-buttress-342908'
dataset_id = 'primal-buttress-342908.test'
table_id='test3'

df = get_estat_dataframe(params)
# データフレームの内容をBigQueryのテーブルに追加
df.to_gbq( f'{dataset_id}.{table_id}', project_id=project_id, if_exists="append")

# 統計表に含まれる全てのデータをDataframeに変換する

'statsDataId'で統計表を指定して、含まれる統計データを一括でDataFrameに変換する

In [ ]:
statsDataId = '0000010101'

In [ ]:
def get_estat_dataframe_all(statsDataId):

  # 統計表のメタ情報を取得
  meta = get_estat_meta(statsDataId)

  # カテゴリ一覧
  category_codes = get_estat_catefory_codes(statsDataId)

  # 地域コード一覧
  area_codes = get_estat_area_codes(statsDataId)

  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  for category_code in category_codes:
    # リクエストパラメータ生成
    params = {}
    params['statsDataId'] = statsDataId
    params['cdCat01'] = category_code
    params['cdArea']=",".join(area_codes)

    # DataFrameを取得
    print(category_code)
    df_category = get_estat_dataframe(params)

    df_res = pd.concat([df_res, df_category])

  # return df_res
  return 'test'


df = get_estat_dataframe_all(statsDataId)
# df


# 新しいセクション